# Masking T1s for plots

Take the subject T1s and apply the mask image, to make them prettier for plots

In [1]:
from bids.layout import BIDSLayout
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
maskT1 = pe.Workflow(name="maskT1") # workflow to run the analysis

# Data grabbers and sinkers

In [3]:
# THIS CODE DEFINITELY WORKS
dg = pe.Node(DataGrabber(infields=['sub'], 
                         outfields=['T1','mask']), name='dg')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'T1': [['sub','sub']],
                           'mask':[['sub','sub']]}
dg.inputs.field_template = {'T1': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/anat/sub-%s_desc-preproc_T1w.nii.gz',
                            'mask': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-%s/anat/sub-%s_desc-brain_mask.nii.gz'}

In [4]:
# debugging
dg.inputs.sub = '01'
res = dg.run()
res.outputs

200915-19:36:34,663 nipype.workflow INFO:
	 [Node] Setting-up "dg" in "/tmp/tmpt31kfpcy/dg".
200915-19:36:34,670 nipype.workflow INFO:
	 [Node] Running "dg" ("nipype.interfaces.io.DataGrabber")
200915-19:36:34,677 nipype.workflow INFO:
	 [Node] Finished "dg".



T1 = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/anat/sub-01_desc-preproc_T1w.nii.gz
mask = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-01/anat/sub-01_desc-brain_mask.nii.gz

In [5]:
# THIS WORKS
info = pe.Node(ni.IdentityInterface(fields=['sub']),
                   name='info')
info.iterables = [('sub', ['01', '02', '03', '04', '05'])]
# info.iterables = [('sub', ['03'])]
info.iterables 

[('sub', ['01', '02', '03', '04', '05'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0]
    return name

## Data sink

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/"
substitutions = [('_sub_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

## ApplyMask 

In [8]:
mask = pe.Node(fsl.ApplyMask(), name='mask')

In [9]:
def printOutFile(fileName):
    import os
    import re
    fname = os.path.normpath(fileName)
    l = fname.split(os.sep)
    fname = l[-1]
    fname = fname.replace('preproc', 'masked')
    return fname

## Connect workflow

In [10]:
maskT1.connect([(info, dg, [('sub', 'sub')]),
                (dg, ds, [(('T1', printSubPath), 'container')]),
                (dg, mask, [('T1', 'in_file')]),
                (dg, mask, [('mask', 'mask_file')]),
                (dg, mask, [(('T1', printOutFile), 'out_file')]),
                (mask, ds, [('out_file', 'anat.@ANAT')])
             ])
maskT1.run()

200915-19:36:38,890 nipype.workflow INFO:
	 Workflow maskT1 settings: ['check', 'execution', 'logging', 'monitoring']
200915-19:36:38,915 nipype.workflow INFO:
	 Running serially.
200915-19:36:38,916 nipype.workflow INFO:
	 [Node] Setting-up "maskT1.dg" in "/tmp/tmpt31kfpcy/dg".
200915-19:36:38,922 nipype.workflow INFO:
	 [Node] Running "dg" ("nipype.interfaces.io.DataGrabber")
200915-19:36:38,928 nipype.workflow INFO:
	 [Node] Finished "maskT1.dg".
200915-19:36:38,929 nipype.workflow INFO:
	 [Node] Setting-up "maskT1.mask" in "/tmp/tmpjziqp1qw/maskT1/_sub_05/mask".
200915-19:36:38,934 nipype.workflow INFO:
	 [Node] Running "mask" ("nipype.interfaces.fsl.maths.ApplyMask"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-05/anat/sub-05_desc-preproc_T1w.nii.gz -mas /scratch/qbi/uqkgarn1/STRIWP1/derivatives/sub-05/anat/sub-05_desc-brain_mask.nii.gz sub-05_desc-masked_T1w.nii.gz
200915-19:36:41,829 nipype.workflow INFO:
	 [Node] Finished "maskT1